In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)

### 텍스트 분류 실습 - 20 뉴스그룹 분류

In [3]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [4]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [5]:
import pandas as pd

print('target 클래스의 값과 분포도\n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들\n', news_data.target_names)

target 클래스의 값과 분포도
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [7]:
# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target
print(type(X_train), len(X_train))

<class 'list'> 11314


In [8]:
print(X_train[0])



What I did NOT get with my drive (CD300i) is the System Install CD you
listed as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120
from Direct Express in Chicago (no complaints at all -- good price & good
service).

BTW, I've heard that the System Install CD can be used to boot the mac;
however, my drive will NOT accept a CD caddy is the machine is off.  How can
you boot with it then?

--Dave



In [9]:
print(y_train[0])

4


In [10]:
# subset='test'으로 테스트 데이터만 추출,
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

In [11]:
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


In [12]:
y_test[:5]

array([ 4, 11,  1,  7,  8])

### 피쳐 벡터화 변환과 머신러닝 모델 학습/예측/평가

- CountVectorizer

In [13]:
from sklearn. feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

In [14]:
X_train_cnt_vect.shape

(11314, 101631)

In [15]:
# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피터 벡터화 변환 수행.
X_test_cnt_vect = cnt_vect.transform(X_test)

In [15]:
print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


/home/ryung/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
pred[:5]

array([ 4, 11,  6,  7,  8])

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)


In [18]:
# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.674


In [19]:
pred[:5]

array([5, 1, 1, 7, 8])

In [20]:
tfidf_vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [21]:
lr_clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [23]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

In [24]:
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


In [25]:
pred[:5]

array([ 3, 11,  1,  7,  8])

In [26]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

/home/ryung/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


## SVM - TFIDF

In [20]:
from sklearn.svm import SVC

svm = SVC()

svm.fit(X_train_tfidf_vect,y_train)
svm_pred = svm.predict(X_test_tfidf_vect)

In [21]:
svm_pred[:5]

array([3, 7, 1, 7, 8])

In [23]:
from sklearn.metrics import accuracy_score
print('TF-IDF Vectorized SVM 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, svm_pred)))

TF-IDF Vectorized SVM 예측 정확도는 0.678


## LGBMClassifier - TFIDF

In [32]:
from lightgbm import LGBMClassifier
lgb_reg = LGBMClassifier(n_estimators=1000)

lgb_reg.fit(X_train_tfidf_vect,y_train)
lgb_pred = lgb_reg.predict(X_test_tfidf_vect)

In [33]:
lgb_pred[:5]

array([3, 9, 8, 4, 8])

In [34]:
print('TF-IDF Vectorized LGBMRegressor 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, lgb_pred)))

TF-IDF Vectorized LGBMRegressor 예측 정확도는 0.556


## XGBRegressor - TFIDF

In [44]:
from xgboost import XGBClassifier

xgb_reg = XGBClassifier(n_estimator=1000)
xgb_reg.fit(X_train_tfidf_vect,y_train)
xgb_pred = xgb_reg.predict(X_test_tfidf_vect)

[00:40:40] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [45]:
xgb_pred[:5]

array([ 3, 18,  1,  7,  8])

In [46]:
print('TF-IDF Vectorized XGBRegressor 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, xgb_pred)))

TF-IDF Vectorized XGBRegressor 예측 정확도는 0.603


## RandomForestClassifier - TFIDF

In [38]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_tfidf_vect, y_train)
rf_pred = rf_clf.predict(X_test_tfidf_vect)

In [39]:
rf_pred[:5]

array([3, 8, 1, 7, 8])

In [40]:
print('TF-IDF Vectorized RandomForestClassifier 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, rf_pred)))

TF-IDF Vectorized RandomForestClassifier 예측 정확도는 0.634


## SVM - CountVec

In [47]:
from sklearn.svm import SVC

svm = SVC()

svm.fit(X_train_cnt_vect,y_train)
svm_pred = svm.predict(X_test_cnt_vect)

In [49]:
svm_pred[:5]

array([6, 6, 6, 6, 6])

In [50]:
print('Count Vectorized SVM 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, svm_pred)))

Count Vectorized SVM 예측 정확도는 0.097


## SVM - TFIDF (GridSearchCV)

- case 1

In [34]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
grid_svc = GridSearchCV(svc, parameters, refit=True)
grid_svc.fit(X_train_tfidf_vect, y_train)
# GridSearchCV(estimator=SVC(), param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [35]:
scores_df = pd.DataFrame(grid_svc.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,99.762544,6.347715,12.963048,0.675728,1,linear,"{'C': 1, 'kernel': 'linear'}",0.750331,0.733540,0.748564,0.756960,0.742706,0.746420,0.007883,1
1,103.559969,4.198677,13.325650,0.829959,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.695537,0.677861,0.687141,0.686699,0.689655,0.687379,0.005707,3
2,92.938886,3.177920,12.280730,0.556285,10,linear,"{'C': 10, 'kernel': 'linear'}",0.674326,0.665930,0.664163,0.698630,0.642352,0.669080,0.018162,4
3,101.733643,1.289272,13.098522,0.280551,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.730446,0.708794,0.726027,0.732656,0.724580,0.724501,0.008377,2


In [36]:
print('GridSearch 최적 파라미터:', clf.best_params_)
print('GridSearch 최고 정확도:{0:.4f}'.format(clf.best_score_))

GridSearch 최적 파라미터: {'C': 1, 'kernel': 'linear'}
GridSearch 최고 정확도:0.7464


- case 2

In [42]:
parameters = {'kernel':['poly'], 'C':[1, 10]}
svc = svm.SVC()
grid_svc = GridSearchCV(svc, parameters, refit=True)
grid_svc.fit(X_train_tfidf_vect, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ['poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [43]:
scores_df = pd.DataFrame(grid_svc.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,101.549393,1.202430,14.011813,0.260504,1,poly,"{'C': 1, 'kernel': 'poly'}",0.264693,0.247459,0.253646,0.252320,0.291335,0.261891,0.015765,1
1,101.590408,0.779823,13.899724,0.167185,10,poly,"{'C': 10, 'kernel': 'poly'}",0.124613,0.114450,0.125055,0.150685,0.119805,0.126922,0.012488,2


- case 3

In [ ]:
parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[0.1, 1, 10, 100], 'degree':[1,2,3,4,5]}
svc = svm.SVC()
grid_svc = GridSearchCV(svc, parameters, refit=True)
grid_svc.fit(X_train_tfidf_vect, y_train)

In [ ]:
scores_df = pd.DataFrame(grid_svc.cv_results_)
scores_df